In [3]:
import matplotlib.pyplot as plt
import itertools
import functools
import statistics
import math

plt.rc('image', cmap='viridis')
plt.style.use('dark_background')


def Input(day):
    "Open input file for the day"
    file = '2022/input{0}.txt'.format(day)
    return open(file)
